In [ ]:
from typing import List
from langchain.schema import Document
import re
import string
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
from collections import Counter
import networkx as nx
import matplotlib.pyplot as plt
import spacy
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import numpy as np
from PIL import Image
import spacy
import plotly.express as px
import seaborn as sns
nltk.download('punkt')
nltk.download('stopwords')

nlp = spacy.load("pl_core_news_sm")

In [ ]:
tekst = """
Interpretacja indywidualna
– stanowisko nieprawidłowe
Szanowni Państwo,
stwierdzam, że Państwa stanowisko w sprawie oceny skutków podatkowych opisanego zdarzenia przyszłego w podatku od towarów i usług jest nieprawidłowe.
Zakres wniosku o wydanie interpretacji indywidualnej
28 lipca 2025 r. wpłynął Państwa wniosek z 28 lipca 2025 r. o wydanie interpretacji indywidualnej dotyczący opodatkowania paczek wydanych w ramach kampanii reklamowej. Uzupełnili go Państwo – w odpowiedzi na wezwanie – pismem z 4 września 2025 r. (wpływ 4 września 2025 r.).
Treść wniosku jest następująca:
Opis zdarzenia przyszłego
Wnioskodawca w swojej działalności prowadzi sprzedaż towarów, kosmetyków i wyrobów medycznych oraz przeprowadza szkolenia z zakresu zabiegów kosmetycznych.
Wnioskodawca zamierza przeprowadzić kampanię reklamową polegającą na przekazywaniu paczek z produktami o wartości do 700 zł brutto. Paczki z produktami mają być opatrzone logo firmy i przekazane do losowo wybranych osób fizycznych, które nie są jego klientami ani kontrahentami. Na zawartość paczki mają składać się pełnowartościowe towary handlowe i dodatkowo gadżety takie jak np. klamra do włosów, pomadka itd. Celem jest promocja firmy, zwiększenie świadomości marki oraz pozyskiwanie przyszłych klientów.
Uzupełnienie i doprecyzowanie opisu zdarzenia przyszłego
Na pytanie 1: Czy są i będą Państwo na dzień przekazania paczek z produktami zarejestrowani jako czynny podatnik podatku od towarów i usług, odpowiedzieli Państwo: Tak.
Na pytanie 2: Czy paczki z produktami będą przez Państwa przekazywane nieodpłatnie, odpowiedzieli Państwo: Tak.
Na pytanie 3: Czy w zamian za przekazanie paczek otrzymają Państwo jakiekolwiek wynagrodzenie lub osoby obdarowane zobowiązane będą do wykonania określonych czynności? Jeśli tak, proszę wskazać jakich czynności, odpowiedzieli Państwo: Nie.
Na pytanie 4: Czy w odniesieniu do przekazywanych towarów przysługiwało Państwu/będzie Państwu przysługiwało prawo do obniżenia kwoty podatku należnego o kwotę podatku naliczonego z tytułu ich nabycia, importu lub wytworzenia tych towarów lub ich części składowych, odpowiedzieli Państwo: Tak.
Na pytanie 5: Czy przekazanie paczek z produktami w ramach kampanii reklamowej nastąpi wyłącznie na cele związane z Państwa działalnością gospodarczą, odpowiedzieli Państwo: Tak.
Na pytanie 6: Czy przekazywane produkty będą związane z prowadzoną przez Państwa działalnością gospodarczą? W jaki sposób, odpowiedzieli Państwo: Tak, są to towary handlowe nabywane w celu dalszej odsprzedaży. Dodatkowo paczki będą zawierały gadżety takie jak klamra do włosów, pomadka. Z naszych towarów handlowych i dodatkowych gadżetów będzie przygotowywana paczka PR-owa.
Na pytanie 7: Jaką funkcję pełnią przekazywane produkty, np. funkcję promocyjną, informacyjną, itp., odpowiedzieli Państwo: Przekazywane produkty mają pełnić funkcję promocyjną, zwiększyć świadomość marki i pozyskać przyszłych klientów.
Na pytanie 8: Czy przekazywane produkty mają wartość konsumpcyjną lub też użytkową dla konsumentów, odpowiedzieli Państwo: Tak.
Na pytanie 9: Czy będą Państwo prowadzili ewidencję pozwalającą na ustalenie tożsamości odbiorców, odpowiedzieli Państwo: Nie.
Na pytanie 10: Czy na paczki z produktami będą się składały prezenty o małej wartości, tj. przekazywane jednej osobie towary:
1) łącznej wartości nieprzekraczającej w roku podatkowym kwoty 100 zł (bez podatku), jeżeli prowadzą Państwo ewidencję pozwalającą na ustalenie tożsamości obdarowanych osób;
2) których przekazania nie ujęto w ewidencji, jeżeli jednostkowa cena nabycia towaru (bez podatku), a gdy nie ma ceny nabycia, jednostkowy koszt wytworzenia, określone w momencie przekazywania towaru, nie przekraczają 20 zł,
odpowiedzieli Państwo: Nie.
Na pytanie 11: Czy produkty przekazywane w paczkach będą stanowiły próbki, przez które należy rozumieć egzemplarz towaru lub jego niewielką ilość, które pozwalają na ocenę cech i właściwości towaru w jego końcowej postaci? Czy ewentualne przekazanie próbki będzie miało na celu promocję tego towaru, odpowiedzieli Państwo: Nie.
Na pytanie 12: Czy przekazywana próbka nie będzie służyła zasadniczo zaspokojeniu potrzeb odbiorcy końcowego w zakresie przekazywanego towaru? A jeśli będzie, to czy zaspokojenie potrzeb tego odbiorcy jest nieodłącznym elementem promocji tego towaru i ma skłaniać tego odbiorcę do zakupu promowanego towaru, odpowiedzieli Państwo: Nie będą to próbki.
"""


In [ ]:
def split_text_by_header_regex(text: str) -> List[Document]:
    """
    Split text into chunk using header
    """
    
    text_lines = [line.strip() for line in text.splitlines() if line.strip()] 
    
    HEADER_PATTERN = r'([A-ZŚĆŻŹŃŁÓĘĄ][^\n]*[^\.\?!:]\s*)$'
    
    chunks_content = []
    current_chunk = []
    
    is_initial_header_gathering = True 
    
    for i, line in enumerate(text_lines):
        
        is_header = re.search(HEADER_PATTERN, line)
        
       
        if is_initial_header_gathering:
            if is_header:
                current_chunk.append(line)
            else:
                is_initial_header_gathering = False
                current_chunk.append(line) 
            
            continue 

        if is_header:
            if current_chunk:
                chunks_content.append('\n'.join(current_chunk))
                current_chunk = []
            
            current_chunk.append(line)
            
        else:
            current_chunk.append(line)

    if current_chunk:
        chunks_content.append('\n'.join(current_chunk))

    documents = [Document(page_content=content.strip()) for content in chunks_content if content.strip()]

    return documents

In [ ]:
chunks = split_text_by_header_regex(tekst)

print(f"Liczba uzyskanych fragmentów (chunks): {len(chunks)}\n")

for i, chunk in enumerate(chunks):
    print(f"--- CHUNK {i+1} (Długość: {len(chunk.page_content)} znaków) ---")
    print(chunk.page_content)
    print("-" * 50)

In [ ]:
# słowa klucze które wzwraca llm z tekstu
slowa_kluczowe_lista = [
    "interpretacja indywidualna", "stanowisko nieprawidłowe", "wniosek",
    "opodatkowanie", "paczki", "kampania reklamowa", "przekazywanie",
    "produkty", "promocja", "działalność gospodarcza",
    "towary handlowe", "gadżety", "funkcja promocyjna",
    "próbki towaru", "promocja towaru"
]

In [ ]:
# Konwersja tekstu do małych liter dla spójności
tekst_lower = tekst.lower()

# Lista docelowa do przechowywania słowników (obiektów)
lista_obiektow_zliczen = []

# Zliczanie i tworzenie obiektu
for fraza in slowa_kluczowe_lista:
    # Zliczanie wystąpień
    liczba_wystapien = tekst_lower.count(fraza)

    # Tworzenie słownika (obiektu) dla danej frazy
    obiekt_zliczenia = {
        "fraza": fraza,
        "liczba_wystapien": liczba_wystapien
    }

    # Dodawanie słownika do listy
    lista_obiektow_zliczen.append(obiekt_zliczenia)

# Wyświetlenie końcowej listy obiektów
import json
print(json.dumps(lista_obiektow_zliczen, indent=4, ensure_ascii=False))


In [ ]:
# ngramy
tokeny = tekst.lower().split()
# tworzenie tabeli tlumaczen
znaki_do_usuniecia = string.punctuation 
tabela_tlumaczen = str.maketrans('', '', znaki_do_usuniecia)

tekst_bez_interpunkcji = tekst_lower.translate(tabela_tlumaczen)
print(f"Tokeny: {tokeny}")

n = 2
ngramy_nltk = list(ngrams(tokeny, n))

print(f"\nGenerowane {n}-gramy (trigramy):")
print(ngramy_nltk)

In [ ]:
# zliczanie wystapien
licznik_bigramow = Counter(ngramy_nltk) 

bigramy_powyzej_1 = {
    para: liczba 
    for para, liczba in licznik_bigramow.items()
    if liczba > 0
}

print("Liczba wystąpień każdej pary w podanej liście:")
for para, liczba in bigramy_powyzej_1.items():
    print(f"{para}: {liczba}")

In [ ]:
lista_krawedzi = []
for (w1, w2), waga in bigramy_powyzej_1.items():
    lista_krawedzi.append((w1, w2, {"weight": waga}))

print(lista_krawedzi)

In [ ]:
# 1. Tworzenie grafu z listy krawędzi
WG = nx.Graph()
WG.add_edges_from(lista_krawedzi) # Tutaj graf pobiera obiekty z listy

# 2. Odwołanie do obiektów (wag) z grafu
# Pobranie wag jako etykiet krawędzi
labels = nx.get_edge_attributes(WG, "weight")

# Ustalenie grubości krawędzi na podstawie wag
wagi = [d['weight'] for (u, v, d) in WG.edges(data=True)]
grubosc_linii = [waga * 0.4 for waga in wagi]

# --- Rysowanie ---
pos = nx.spring_layout(WG, seed=42) # Stały układ dla powtarzalności

# plt.figure(figsize=(20, 18))
plt.figure(figsize=(30, 28))
plt.axis('off')

# Rysowanie węzłów i etykiet
nx.draw_networkx_nodes(WG, pos, node_size=1500, node_color='skyblue', alpha=0.8)
nx.draw_networkx_labels(WG, pos, font_size=10, font_family='sans-serif', font_weight='bold')

# Rysowanie krawędzi z uwzględnieniem wagi (odwołanie do grubosc_linii)
nx.draw_networkx_edges(WG, pos, width=grubosc_linii, edge_color='darkslategray', alpha=0.7)

# Rysowanie etykiet krawędzi (odwołanie do labels)
nx.draw_networkx_edge_labels(WG, pos, edge_labels=labels, font_color='red', font_size=9)

plt.title("Graf częstotliwości par słów", fontsize=16)
plt.show()

In [ ]:
# Ustalenie stałej listy części mowy do uwzględnienia
FILTROWANE_POS = {"NOUN", "VERB", "ADJ"}
OKNO_WSPOLWYSTEPOWANIA = 3 # Okno, w którym słowa są uważane za powiązane

# --- ETAP PRZETWARZANIA TEKSTU I TWORZENIA KRAWĘDZI ---

# Wczytanie modelu języka polskiego
try:
    nlp = spacy.load("pl_core_news_sm")
except OSError:
    print("Błąd: Nie znaleziono modelu 'pl_core_news_sm'. Upewnij się, że został pobrany komendą 'python -m spacy download pl_core_news_sm'")
    exit()

doc = nlp(tekst)
lematy_filtrowane = []

# Filtracja i lematyzacja słów
for token in doc:
    # Kluczowe filtrowanie według części mowy: NOUN, VERB, ADJ
    if token.pos_ in FILTROWANE_POS and not token.is_punct and len(token.text) >= 2:
        lematy_filtrowane.append(token.lemma_)

# Tworzenie krawędzi na podstawie współwystępowania w oknie
krawedzie = []
for i in range(len(lematy_filtrowane)):
    # Sprawdzamy współwystępowanie w obrębie zadanego okna
    for j in range(i + 1, min(i + 1 + OKNO_WSPOLWYSTEPOWANIA, len(lematy_filtrowane))):
        # Normalizacja krawędzi (mniejsze słowo pierwsze) dla łatwiejszego zliczania
        u, v = sorted([lematy_filtrowane[i], lematy_filtrowane[j]])
        # Zapobiegamy pętlom (słowo powiązane z samym sobą)
        if u != v:
            krawedzie.append((u, v))

# Zliczanie częstotliwości (wag) krawędzi
licznik_krawedzi = Counter(krawedzie)

# Tworzenie listy krawędzi w formacie akceptowanym przez networkx (z wagą)
# Użyjemy list krotek (węzeł1, węzeł2, słownik_atrybutów)
lista_krawedzi = [(u, v, {"weight": count}) for (u, v), count in licznik_krawedzi.items() if count > 1] # Filtrujemy rzadkie krawędzie

# --- ETAP WIZUALIZACJI GRAFU (Twój kod) ---

# 1. Tworzenie grafu z listy krawędzi
WG = nx.Graph()
WG.add_edges_from(lista_krawedzi) # Tutaj graf pobiera obiekty z listy

# 2. Odwołanie do obiektów (wag) z grafu
# Pobranie wag jako etykiet krawędzi
labels = nx.get_edge_attributes(WG, "weight")

# Ustalenie grubości krawędzi na podstawie wag
wagi = [d['weight'] for (u, v, d) in WG.edges(data=True)]
grubosc_linii = [waga * 0.4 for waga in wagi] # Skalowanie jest kluczowe

# --- Rysowanie ---
pos = nx.spring_layout(WG, seed=42) # Stały układ dla powtarzalności

plt.figure(figsize=(30, 28))
plt.axis('off')

# Rysowanie węzłów i etykiet
nx.draw_networkx_nodes(WG, pos, node_size=1500, node_color='skyblue', alpha=0.8)
nx.draw_networkx_labels(WG, pos, font_size=10, font_family='sans-serif', font_weight='bold')

# Rysowanie krawędzi z uwzględnieniem wagi (odwołanie do grubosc_linii)
nx.draw_networkx_edges(WG, pos, width=grubosc_linii, edge_color='darkslategray', alpha=0.7)

# Rysowanie etykiet krawędzi (odwołanie do labels)
nx.draw_networkx_edge_labels(WG, pos, edge_labels=labels, font_color='red', font_size=9)

plt.title(f"Graf Współwystępowania (Rzeczowniki, Czasowniki, Przymiotniki, okno={OKNO_WSPOLWYSTEPOWANIA})", fontsize=20)
plt.show()

In [ ]:
polish_stopwords = set(STOPWORDS)
polish_stopwords.update(['jest', 'są', 'się', 'dla', 'aby', 'które', 'jakie', 'w', 'ul', 'o', 'że', 'b', 'z', 'od'])

# utworzenie obiektu wordcloud
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    stopwords=polish_stopwords,
    min_font_size=10,
    font_path='arial.ttf' 
).generate(tekst.lower())


plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Chmura Słów na podstawie tekstu")
plt.show()

In [ ]:
# przetwarzanie tekstu i zliczanie slow wg czesci mowy
doc = nlp(tekst)

POLSKIE_POS = {
    "ADJ": "Przymiotnik",
    "ADP": "Przyimek",
    "ADV": "Przysłówek",
    "AUX": "Czasownik posiłkowy",
    "CONJ": "Spójnik",
    "CCONJ": "Spójnik współrzędny",
    "DET": "Zaimek",
    "INTJ": "Wykrzyknik",
    "NOUN": "Rzeczownik",
    "NUM": "Liczebnik",
    "PART": "Partykuła",
    "PRON": "Zaimek",
    "PROPN": "Nazwa własna",
    "PUNCT": "Interpunkcja",
    "SCONJ": "Spójnik podrzędny",
    "SYM": "Symbol",
    "VERB": "Czasownik",
    "X": "Inne",
    "SPACE": "Spacja"
}

pos_data = []

for token in doc:
    # filtrujemy nieistotne elementy i krotkie slowa
    if token.is_stop or token.is_punct or token.is_space or len(token.text) < 3:
        continue

    angielski_pos = token.pos_
    polski_pos = POLSKIE_POS.get(angielski_pos, angielski_pos)
    
    # Zbieramy czesc mowy + lemat
    # pos_data.append((token.pos_, token.lemma_))
    pos_data.append((polski_pos, token.lemma_))


# Zliczanie częstotliwości występowania dla każdej pary (POS, Lemat)
pos_counts = Counter(pos_data)

# 4. Przygotowanie danych dla wykresu
df = pd.DataFrame([
    {
        'Część Mowy': pos,
        'Lemat': lemat,
        'Waga': waga
    }
    for (pos, lemat), waga in pos_counts.items()
])

unikalne_pos = df['Część Mowy'].unique()
unikalne_pos = unikalne_pos[(unikalne_pos != 'X') & (unikalne_pos != 'NUM')]

LIMIT_SLOW = 50 

for pos_category in unikalne_pos:
    df_filtered = df[df['Część Mowy'] == pos_category].sort_values(by='Waga', ascending=False).head(LIMIT_SLOW)
    
    if df_filtered.empty:
        continue

    # Ustawienie MNIEJSZEJ szerokości (np. 8 cali zamiast 10)
    # Zmniejszenie współczynnika wysokości (np. z 0.4 do 0.3 cala na słowo)
    NOWA_SZEROKOSC = 7
    NOWA_WYSOKOSC = len(df_filtered) * 0.3 + 1 
    
    plt.figure(figsize=(NOWA_SZEROKOSC, NOWA_WYSOKOSC)) 
    
    sns.barplot(
        x='Waga', 
        y='Lemat', 
        data=df_filtered,
        color=sns.color_palette("viridis")[0]
    )

    plt.title(f'TOP {LIMIT_SLOW} Najczęściej Występujących Słów: {pos_category}')
    plt.xlabel('Częstotliwość (Waga)')
    plt.ylabel('Słowo (Lemat)')
    
    plt.xlim(0, max(df_filtered['Waga']) * 1.1)
    
    plt.tight_layout()
    plt.show()